In [13]:
import yfinance as yf

ticker = yf.Ticker("AAPL.DE")
df = ticker.history(period="30d", interval="1d")
print(len(df))
print(df)

$AAPL.DE: possibly delisted; no price data found  (period=30d)


0
Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
